In [103]:
import psycopg2
from psycopg2.extras import execute_values
import configparser
import csv

In [109]:

config = configparser.ConfigParser()
config.read('../config.ini')

books_csv_url = config['URLIntermediate']['books']
users_csv_url = config['URLIntermediate']['users']
ratings_csv_url = config['URLIntermediate']['ratings']

rds_db_endpoint = config['AWS']['rds_db_endpoint']
rds_db_port = config['AWS']['rds_db_port']
rds_db_user = config['AWS']['rds_db_user']
rds_db_password = config['AWS']['rds_db_password']
rds_db_name = config['AWS']['rds_db_name']

In [105]:

try:
    # conn = psycopg2.connect("host= localhost port=5432  user=postgres  password=anamac")
    conn = psycopg2.connect(f"host={rds_db_endpoint} port={rds_db_port}  user={rds_db_user}  password={rds_db_password}")
except psycopg2.Error as e :
    print(e)

In [106]:
try:
    cur = conn.cursor()
except psycopg2.Error as e :
    print(e)

In [107]:
conn.set_session(autocommit=True)

In [108]:

try:
    cur.execute(f"CREATE DATABASE {rds_db_name}")
except psycopg2.Error as e :
    print(e)

In [110]:
try:
    conn = psycopg2.connect(f"host={rds_db_endpoint} port={rds_db_port}  user={rds_db_user}  password={rds_db_password} dbname={rds_db_name}")
    # conn = psycopg2.connect("host= localhost port=5432  user=postgres  password=anamac dbname=db_book_recommendation")
except psycopg2.Error as e :
    print(e)

In [111]:
try:
    cur = conn.cursor()
    conn.set_session(autocommit=True)
except psycopg2.Error as e :
    print(e)
    

In [112]:
sql = '''
    CREATE TABLE USERS(
        userID integer PRIMARY KEY,
        age integer,
        location_city varchar(255),
        location_department varchar(255),
        location_country varchar(255)
    )
'''
try:
    cur.execute(sql)
except psycopg2.Error as e :
    print(e)

In [113]:
data_users = []
with open(users_csv_url, 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader :
        data_users.append((row[0], row[1], row[2], row[3], row[4]))


In [114]:
try:
    execute_values(cur, "INSERT INTO USERS(userID, age, location_city, location_department, location_country) VALUES %s", data_users)
except psycopg2.Error as e :
    print(e)

In [115]:
sql3 = '''
    CREATE TABLE RATINGS(
        userID integer ,
        ISBN char(15),
        book_rating integer,
        PRIMARY KEY(userID, ISBN)
    )
'''
try:
    cur.execute(sql3)
except psycopg2.Error as e :
    print(e)

In [116]:

with open(ratings_csv_url, 'r') as f:
    reader = csv.reader(f)
    next(reader)
    data_ratings = [(row[0], row[1], row[2]) for row in reader]

In [117]:
try:
    execute_values(cur, "INSERT INTO RATINGS(userID, ISBN, book_rating) VALUES %s", data_ratings)
except psycopg2.Error as e :
    print(e)

In [ ]:
sql4 = '''
    CREATE TABLE BOOKS(
        ISBN char(15) PRIMARY KEY,
        book_title varchar(450),
        book_author varchar(255),
        year_of_publication integer,
        publisher varchar(255)
    )
'''
try:
    cur.execute(sql4)
except psycopg2.Error as e :
    print(e)

In [ ]:

with open(books_csv_url, 'r') as f:
    reader = csv.reader(f)
    next(reader)
    data_books = [(row[0], row[1], row[2], row[3], row[4]) for row in reader]

In [ ]:
try:
    execute_values(cur, '''INSERT INTO BOOKS(  
                   ISBN ,
        book_title ,
        book_author ,
        year_of_publication, 
        publisher ) VALUES %s''', data_books)
except psycopg2.Error as e :
    print(cur)
    print(e)